用paddle.vision.models下的restnet50网络。把图片都裁剪成867X1155，主要为了减小图片尺寸，去掉周边对无用的图片信息。采用INNER_AREA插值进行resize，减小失真。momentem,0.9,学习率0.0001，是多次实验之后的选择。800张图片都用来训练，patch=10,训练75轮后提交后AUC：0.99962
忘记保留checkpoint

In [ ]:
#安装xlwt，用于生成xls表格，保存预测结果
!pip install xlwt

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 102kB 10.6MB/s ta 0:00:01


In [4]:
#切换到work目录下运行
%cd work
import paddle
import os
import cv2
import numpy as np
from paddle.vision.models import resnet101,resnet50
from paddle.nn import Linear
import paddle.nn.functional as F
from paddle.optimizer import Momentum
from paddle.regularizer import L2Decay
from paddle.nn import CrossEntropyLoss
from paddle.metric import Accuracy, Auc
import matplotlib.pyplot as plt
from paddle.vision.transforms import RandomHorizontalFlip,RandomVerticalFlip
from PIL import ImageEnhance,Image
import random
import xlwt
import pandas as pd


#用于图像裁剪，裁剪成H X W 是867X1155的图片
def image_crop(img):
    H=img.shape[0]
    img_crop=img[int(H*0.2):int(H*0.8),0:int(H*0.8)]
    img_crop_H=img_crop.shape[0]
    img_crop_W=img_crop.shape[1]
    return img_crop,img_crop_H,img_crop_W



#获取图片信息，包括图片名字和标签，以列表的形式，列表里面包含一个个元组，格式是（图片名字，标签）
def get_annotations(mode1=None):
    df = pd.read_excel('Train/Classification.xlsx')
    images = df.loc[:,'imgName'].values
    labels = df.loc[:,'Label'].values
    anno_lst=[]
    test=[]
    for image,label in zip(images,labels):
        anno_lst.append((image,label))
    if mode1 =='train':
        result=anno_lst[0:800]       

    elif mode1 =='valid':
        result=anno_lst[700:800]
    elif mode1 =='test':
        images_name=os.listdir('PALM-Testing400-Images')
        for image in images_name:
            test.append((image,1000))
        return test
    
    return result


#获取图象数据，返回的值分别是图像数组，标签，图片名字
def get_img_data(datadir,record, size, mode2=None):
    
    img=cv2.imread(datadir+'/'+record[0])
    img,H,W=image_crop(img)
    if H!=867:
        img=cv2.resize(img,size,interpolation=cv2.INTER_AREA)
    
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    mean = np.array(mean).reshape((1, 1, -1))
    std = np.array(std).reshape((1, 1, -1))
    img = (img / 255.0 - mean) / std
    
    img=np.transpose(img,(2,0,1)).astype('float32')
    
    
    if mode2=='train' or 'valid':
        record1=np.array([record[1]])
        record1=record1.astype('int64')
    return img,record1,record[0]
    



class TrainDataset(paddle.io.Dataset):
    def  __init__(self,mode='train'):
        if mode=='train' or mode=='valid':
            self.datadir='Train/fundus_image'
        elif mode=='test':
            self.datadir='PALM-Testing400-Images'
    
        self.mode=mode
        self.records = get_annotations(mode1=mode)
        
        self.img_size = (1155, 867)  #get_img_size(mode)

    def __getitem__(self, idx):
        record = self.records[idx]
        # print("print: ", record)
        try:
            
            img, label,img_name = get_img_data(self.datadir,record, size=self.img_size,mode2=self.mode)
                                
        except:
            print('图片出现错误',record)

        return img, label,img_name
    def __len__(self):
        return len(self.records)



#定义模型主体
class resnet_model(paddle.nn.Layer):
    def __init__(self,class_num):
        super(resnet_model,self).__init__()
        self.model=resnet50(pretrained=True)
        self.fc=Linear(1000,class_num)
    def forward(self,x):
        out=self.model(x)
        out1=self.fc(out)
        return out1



#训练函数
def train(model):
    
    opt = paddle.optimizer.Momentum(learning_rate=0.0001, momentum=0.9, parameters=model.parameters(), weight_decay=0.001)
    
    train_dataset=TrainDataset(mode='train')
    train_loader=paddle.io.DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=0, drop_last=True)
    valid_dataset=TrainDataset(mode='valid')
    valid_loader=paddle.io.DataLoader(valid_dataset, batch_size=10, shuffle=True, num_workers=0, drop_last=True)
    epoches=76
    model.train()
    
    for epoch in range(epoches):

        for batch_id,batch in enumerate(train_loader()):
            x=batch[0]
            label=batch[1]
            out=model(x)
            
            
            
            loss = F.cross_entropy(out,label)
            
            
            

            if batch_id % 5 == 0:
                print("train:epoch: {}, batch_id: {}, loss is: {:.5f}".format(epoch, batch_id, loss.numpy()[0]))
            # 反向传播，更新权重，清除梯度
            loss.backward()
            opt.step()
            opt.clear_grad()

        paddle.save(model.state_dict(), 'output/epoch'+str(epoch)+'.pdparams')
        paddle.save(opt.state_dict(), 'output/epoch'+str(epoch)+'.pdopt')

        
'''
        model.eval()
        accuracies = []
        losses = []
        
        for batch_id, data in enumerate(valid_loader()):
            img=data[0]
            label1=data[1]
           
            # 运行模型前向计算，得到预测值
            out1 = model(img)
            # 二分类，sigmoid计算后的结果以0.5为阈值分两个类别
            # 计算sigmoid后的预测概率，进行loss计算
            
            pred = F.softmax(out1)
            loss = F.cross_entropy(out1, label1)
           
            
            acc = paddle.metric.accuracy(pred, label1)

            accuracies.append(acc.numpy())
            losses.append(loss.numpy())
            print("[validation] accuracy/loss: {:.5f}/{:.5f}".format(acc.numpy()[0], loss.numpy()[0]))
        
        model.train()

 '''       


        

#预测函数
def predict(model):
   
    #预测时，请加载训练好的用来预测的xxx.pdparams
    model_pararams=paddle.load('output/75_99962.pdparams')
    

    model.load_dict(model_pararams)

    model.eval()
    
    test_dataset=TrainDataset(mode='test')
    test_loader=paddle.io.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=True)
    
    
    m = Auc()
    
    num=0
    nums=len(test_loader())
    sum0=[]
    preds=[]
    labels=[]
    book = xlwt.Workbook() #创建Excel
    sheet = book.add_sheet('sheet1')
    sheet.write(0,0,'FileName')
    sheet.write(0,1,'PM Risk')

    for batch_id,batch in enumerate(test_loader()):
        
        x=batch[0]
        
        out=model(x)
        
        result=F.softmax(out)
        result=result.numpy()
        preds.append(list(result[0]))
        #labels.append(batch[1].numpy()[0])
        #if np.argmax(result[0])==batch[1].numpy()[0][0]:
        #    num=num+1
        #if batch[1].numpy()[0][0]==1:
        #    sum0.append(result[0][1])
        #elif batch[1].numpy()[0][0]==0:
        #    sum0.append(result[0][0])
        sheet.write(batch_id+1,0,batch[2])
        sheet.write(batch_id+1,1,float(result[0][1]))
                
        book.save('Classification_Results.xls')
        #print(batch[2],classes[np.argmax(result[0])],result[0].max())
    #print('评价指标1，valid上的准确率：',num,nums,num/nums)
    #print('评价指标2,valid上的准确率:',sum(sum0)/len(sum0))
    #preds=np.array(preds)
    #labels=np.array(labels)
    
    #m.update(preds=preds, labels=labels)
    #res = m.accumulate()
    #print('Auc:',res)
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')



model1 = resnet_model(2)


#启动训练，预测时请注释
#train(model1)

#启动预测，训练时请注释
predict(model1)

[Errno 2] No such file or directory: 'work'
/home/aistudio/work
